[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github.com/RichmondAlake/agent_memory_course/blob/main/mem0/memory_augmented_agent_with_mem0_mongodb.ipynb)


In [1]:
%pip install -qU mem0ai azure-identity

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 常量
CURRENT_USER_ID = "user-123"

In [3]:
import getpass
import os

try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  # 无 python-dotenv 时使用系统环境变量

# 安全地获取并设置环境变量的函数
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [4]:
# set_env_securely("OPENAI_API_KEY", "Enter your OPENAI_API_KEY: ")

In [5]:
# set_env_securely("MONGODB_URI", "Enter your MongoDB URI: ")

In [6]:
# config = {
#     "vector_store": {
#         "provider": "mongodb",
#         "config": {
#             "db_name": "mem0_agent_memory",
#             "collection_name": "extracted_memories",
#             "mongo_uri": os.environ["MONGODB_URI"]
#         }
#     }
# }
from langchain_voyageai import VoyageAIEmbeddings

config = {
    "vector_store": {
        "provider": "mongodb",
        "config": {
            "db_name": "mem0_agent_memory",
            "collection_name": "extracted_memories",
            "mongo_uri": os.environ["MONGODB_URI"],
            "embedding_model_dims": 1024
        }
    },
    "embedder": {
        "provider": "langchain",
        "config": {
            "model": VoyageAIEmbeddings(
                model="voyage-3-large",
                voyage_api_key=os.getenv("VOYAGE_API_KEY")
            ),
            "embedding_dims": 1024
        }
    },
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": "gpt-4o",
            "azure_kwargs": {
                "api_key": os.getenv("OPENAI_API_KEY"),
                "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
                "azure_deployment": "gpt-4o",
                "api_version": "2025-03-01-preview"
            }
        }
    }
}

# semantic_memory_config = {
#     "vector_store": {
#         "provider": "mongodb",
#         "config": {
#             "db_name": "mem0_agent_memory",
#             "collection_name": "semantic_memory",
#             "mongo_uri": os.environ["MONGODB_URI"]
#         }
#     }
# }

/Users/binzhou/Demo/agent_memory_demo/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# 在 semantic_memory_config（以及需要用到 Voyage 的 config）里：
semantic_memory_config = {
    "vector_store": {
        "provider": "mongodb",
        "config": {
            "db_name": "mem0_agent_memory",
            "collection_name": "semantic_memory",
            "mongo_uri": os.environ["MONGODB_URI"],
            "embedding_model_dims": 1024
        }
    },
    "embedder": {
        "provider": "langchain",
        "config": {
            "model": VoyageAIEmbeddings(
                model="voyage-3-large",
                voyage_api_key=os.getenv("VOYAGE_API_KEY")
            ),
            "embedding_dims": 1024
        }
    }
}

In [8]:
# 若曾用 1536 维（如 OpenAI）建过索引，需先删掉旧索引，再跑下面 Memory.from_config 会按 1024 维（Voyage）重建。
# 只需在出现 "vector field is indexed with 1536 dimensions but queried with 1024" 时运行一次即可。
from pymongo import MongoClient
_client = MongoClient(os.environ["MONGODB_URI"])
_db = _client["mem0_agent_memory"]
for _coll_name in ["extracted_memories", "semantic_memory"]:
    _idx_name = f"{_coll_name}_vector_index"
    try:
        _db[_coll_name].drop_search_index(_idx_name)
        print(f"Dropped search index '{_idx_name}'.")
    except Exception as e:
        if "index not found" in str(e).lower() or "no such index" in str(e).lower():
            print(f"Index '{_idx_name}' not found (ok).")
        else:
            print(f"Drop '{_idx_name}': {e}")
_client.close()

Dropped search index 'extracted_memories_vector_index'.
Drop 'semantic_memory_vector_index': Search index mem0_agent_memory.semantic_memory.semantic_memory_vector_index cannot be found, full error: {'ok': 0.0, 'errmsg': 'Search index mem0_agent_memory.semantic_memory.semantic_memory_vector_index cannot be found', 'code': 27, 'codeName': 'IndexNotFound', '$clusterTime': {'clusterTime': Timestamp(1770115086, 1), 'signature': {'hash': b'i\x8c\xa7\xdco\x99\x8e\x11N-\xad\x90 6\x906p\x97hO', 'keyId': 7584727213249921030}}, 'operationTime': Timestamp(1770115086, 1)}


In [21]:
# 若报错 "Index 'extracted_memories_vector_index' does not exist"，先运行本 cell 创建索引，再重新运行下面的 Memory.from_config
from pymongo import MongoClient
from pymongo.operations import SearchIndexModel

_uri = os.environ.get("MONGODB_URI")
if _uri:
    _c = MongoClient(_uri)
    _db = _c["mem0_agent_memory"]
    _dims = 1024
    for _coll_name in ["extracted_memories", "semantic_memory"]:
        _col = _db[_coll_name]
        _idx_name = f"{_coll_name}_vector_index"
        _existing = list(_col.list_search_indexes(name=_idx_name))
        if _existing:
            print(f"Index '{_idx_name}' already exists.")
        else:
            _col.create_search_index(SearchIndexModel(
                name=_idx_name,
                definition={
                    "mappings": {
                        "dynamic": False,
                        "fields": {
                            "embedding": {"type": "knnVector", "dimensions": _dims, "similarity": "cosine"}
                        }
                    }
                }
            ))
            print(f"Created index '{_idx_name}' (may take a minute on Atlas).")
    _c.close()
else:
    print("MONGODB_URI not set.")

Created index 'extracted_memories_vector_index' (may take a minute on Atlas).
Index 'semantic_memory_vector_index' already exists.


In [22]:
from mem0 import Memory
memory = Memory.from_config(config)
semantic_memory = Memory.from_config(semantic_memory_config)

INFO:mem0.vector_stores.mongodb:Search index 'extracted_memories_vector_index' already exists in collection 'extracted_memories'.
INFO:mem0.vector_stores.mongodb:Search index 'mem0migrations_vector_index' already exists in collection 'mem0migrations'.
INFO:mem0.vector_stores.mongodb:Retrieved document with ID 'c0c3c749-98eb-4d92-9620-8283735e0a66'.
INFO:mem0.vector_stores.mongodb:Search index 'semantic_memory_vector_index' already exists in collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Search index 'mem0migrations_vector_index' already exists in collection 'mem0migrations'.
INFO:mem0.vector_stores.mongodb:Retrieved document with ID 'c0c3c749-98eb-4d92-9620-8283735e0a66'.


In [10]:
from openai import OpenAI
from typing import Optional, Dict

# 使用 Azure 时加上这行，否则会走到 Azure 的 tracing 系统，报401Error
os.environ["OPENAI_AGENTS_DISABLE_TRACING"] = "true"

# 导入 OpenAI Python 客户端库
# from openai import OpenAI
from openai import AzureOpenAI

# 初始化 OpenAI 客户端
# 将使用环境变量 OPENAI_API_KEY 中的 API 密钥
#openai_client = OpenAI()
openai_client = AzureOpenAI(
    #api_version="2024-12-01-preview",
    api_version="2025-03-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("OPENAI_API_KEY"),
    azure_deployment="gpt-4o",
)


def patched_search_vector_store(self, query, filters, limit, threshold: Optional[float] = None):
    """
    使用正确 MongoDB 参数名的 _search_vector_store 补丁版本。
    
    mem0 MongoDB search 签名为 (query, vectors, limit, filters)，参数名为 vectors。
    """
    if not query or (isinstance(query, str) and not query.strip()):
        return []
    # 为查询生成嵌入向量
    embeddings = self.embedding_model.embed(query, "search")
    memories = self.vector_store.search(query=query, vectors=embeddings, limit=limit, filters=filters)
    
    # 将结果处理为期望的格式
    promoted_payload_keys = [
        "user_id",
        "agent_id", 
        "run_id",
        "actor_id",
        "role",
    ]

    core_and_promoted_keys = {"data", "hash", "created_at", "updated_at", "id", *promoted_payload_keys}

    # 若指定则应用阈值过滤
    if threshold is not None:
        filtered_memories = []
        for memory_item in memories:
            if hasattr(memory_item, 'score') and memory_item.score >= threshold:
                filtered_memories.append(memory_item)
        memories = filtered_memories

    # 将记忆序列化为期望的格式
    serialized_memories = []
    for memory_item in memories:
        serialized_memory = {}
        
        # 处理核心字段
        for key in core_and_promoted_keys:
            if key == "data":
                # 实际记忆内容在 payload 中
                serialized_memory[key] = getattr(memory_item, 'payload', {}).get('data', None)
            else:
                serialized_memory[key] = getattr(memory_item, key, None)

        # 添加分数和记忆内容
        serialized_memory["score"] = getattr(memory_item, "score", None)
        serialized_memory["memory"] = getattr(memory_item, 'payload', {}).get('data', '')
        serialized_memory["metadata"] = getattr(memory_item, 'payload', {}).get('metadata', {})

        # 添加提升的 payload 键
        payload = getattr(memory_item, 'payload', {})
        for key in promoted_payload_keys:
            value = payload.get(key, None)
            if value is not None:
                serialized_memory[key] = value

        serialized_memories.append(serialized_memory)

    return serialized_memories

def create_patched_add_to_vector_store(original_method):
    """
    工厂函数，用于创建补丁版的 _add_to_vector_store 方法。
    确保每个记忆实例都有自己对原始方法的引用。
    """
    def patched_add_to_vector_store(self, messages, metadata, filters, infer):
        """
        修复 MongoDB 搜索参数问题的 _add_to_vector_store 补丁版本。
        
        该方法在 memory.add() 操作期间被调用，此前也错误地使用了 'vectors' 参数。
        """
        # 从原始模块导入 logger
        import logging
        logger = logging.getLogger(__name__)
        
        # 获取实际的 MongoDB 向量存储搜索方法（非我们的补丁版本）
        mongodb_vector_store = self.vector_store
        
        # 直接获取原始 MongoDB 搜索方法
        original_mongodb_search = mongodb_vector_store.__class__.search
        
        def patched_vector_store_search(self_vs, query, vectors, limit, filters):
            return original_mongodb_search(self_vs, query=query, vectors=vectors, limit=limit, filters=filters)
        
        # 临时补丁向量存储的搜索方法
        original_vector_store_search = mongodb_vector_store.search
        mongodb_vector_store.search = patched_vector_store_search.__get__(mongodb_vector_store, mongodb_vector_store.__class__)
        
        try:
            # 以绑定方法形式调用原始 _add_to_vector_store 方法
            result = original_method.__get__(self, type(self))(messages, metadata, filters, infer)
            return result
        finally:
            # 恢复原始向量存储的搜索方法
            mongodb_vector_store.search = original_vector_store_search
    
    return patched_add_to_vector_store

In [11]:
# 在类和实例两个层级应用猴子补丁
from mem0.memory.main import Memory

# 在打补丁之前保存每个实例的原始方法
memory_original_search = memory._search_vector_store
memory_original_add = memory._add_to_vector_store
semantic_memory_original_search = semantic_memory._search_vector_store
semantic_memory_original_add = semantic_memory._add_to_vector_store

# 在任何补丁之前保存原始类方法
original_class_search = Memory._search_vector_store
original_class_add = Memory._add_to_vector_store

# 应用类级别补丁（影响新实例）
Memory._search_vector_store = patched_search_vector_store
Memory._add_to_vector_store = create_patched_add_to_vector_store(original_class_add)

# 为各实例创建专用的补丁 add 方法
memory_patched_add = create_patched_add_to_vector_store(memory_original_add)
semantic_memory_patched_add = create_patched_add_to_vector_store(semantic_memory_original_add)

# 对已有实例应用补丁
memory._search_vector_store = patched_search_vector_store.__get__(memory, memory.__class__)
memory._add_to_vector_store = memory_patched_add.__get__(memory, memory.__class__)

semantic_memory._search_vector_store = patched_search_vector_store.__get__(semantic_memory, semantic_memory.__class__)
semantic_memory._add_to_vector_store = semantic_memory_patched_add.__get__(semantic_memory, semantic_memory.__class__)

print("🔧 MongoDB search and add methods patched successfully for both memory instances!")

🔧 MongoDB search and add methods patched successfully for both memory instances!


## Data Ingestion

In [12]:
import sys

# 方法 1：将项目根目录添加到 Python 路径
project_root = "/Users/binzhou/Demo/agent_memory_demo/part1"
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utilities.pdf_chunker import ingest_pdf_and_chunk

url = "https://arxiv.org/pdf/2404.13501"

# 摄取并分块 PDF
chunks = ingest_pdf_and_chunk(url)

Loading PDF with LangChain...
Chunking text...
Successfully created 198 chunks


In [13]:
# 将分块逐个摄入语义记忆，避免超时

print(f"📄 Ingesting {len(chunks)} chunks into semantic memory...")

successful_ingestions = 0
failed_ingestions = 0

for i, chunk in enumerate(chunks):
    try:
        # 逐个处理每个分块
        chunk_content = (chunk.get("value") or {}).get("content") or ""
        if not chunk_content or len(chunk_content.strip()) < 50:
            continue
            
        # 为该分块创建单条消息
        message = {"role": "user", "content": chunk_content}
        
        # 添加到语义记忆
        result = semantic_memory.add([message], user_id=CURRENT_USER_ID, infer=False)
        successful_ingestions += 1
        
        if (i + 1) % 10 == 0:  # 每处理 10 个分块更新进度
            print(f"   Processed {i + 1}/{len(chunks)} chunks...")
            
    except Exception as e:
        print(f"   ❌ Failed to ingest chunk {i}: {str(e)[:100]}...")
        failed_ingestions += 1
        continue

print(f"✅ Ingestion complete!")
print(f"   - Successfully ingested: {successful_ingestions} chunks")
print(f"   - Failed: {failed_ingestions} chunks")
print(f"   - Total processed: {successful_ingestions + failed_ingestions}/{len(chunks)}")

📄 Ingesting 198 chunks into semantic memory...


INFO:httpx:HTTP Request: HEAD https://huggingface.co/voyageai/voyage-3-large/resolve/main/tokenizer.json "HTTP/1.1 307 Temporary Redirect"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/voyageai/voyage-3-large/d856baa31d0d467511b9f55f681bd2b17350a46c/tokenizer.json "HTTP/1.1 200 OK"
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongo

   Processed 10/198 chunks...


INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector

   Processed 20/198 chunks...


...

   Processed 180/198 chunks...


INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector

   Processed 190/198 chunks...


INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserting 1 vectors into collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Inserted 1 documents into 'semantic_memory'.
INFO:mem0.vector

✅ Ingestion complete!
   - Successfully ingested: 198 chunks
   - Failed: 0 chunks
   - Total processed: 198/198


In [ ]:
def chat_with_memories(message: str, user_id: str = None) -> str:
    """
    Chat function that retrieves relevant memories and generates responses.
    
    Args:
        message (str): User's input message
        user_id (str): User identifier for memory retrieval (default: CURRENT_USER_ID)
        
    Returns:
        str: Assistant's response based on memories and query
    """
    if user_id is None:
        user_id = CURRENT_USER_ID
    if not message or not message.strip():
        return "Please enter a message."
    try:
        # Retrieve relevant memories using the fixed search method
        relevant_memories = memory.search(query=message, user_id=user_id, limit=3)
        relevant_memories_from_semantic_memory = semantic_memory.search(query=message, user_id=user_id, limit=3)
        
        # Format memories for display and context (defensive: .get("results", []) to avoid KeyError)
        memories_str = "\n".join(f"- {entry['memory']}" for entry in relevant_memories.get("results", []))
        memories_str_from_semantic_memory = "\n".join(f"- {entry['memory']}" for entry in relevant_memories_from_semantic_memory.get("results", []))
        print("Memories:")
        print(memories_str if memories_str else "No relevant memories found.")
        # print("Semantic Memories:")
        # print(memories_str_from_semantic_memory if memories_str_from_semantic_memory else "No relevant semantic memories found.")

        # Generate Assistant response with memory context
        system_prompt = f"You are a helpful AI. Answer the question based on the query and memories.\nUser Memories:\n{memories_str}\nSemantic Memories:\n{memories_str_from_semantic_memory}"
        messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": message}]
        
        response = openai_client.chat.completions.create(model="gpt-4o", messages=messages)
        assistant_response = response.choices[0].message.content

        # Create new memories from the conversation (same user_id 保证下次能检索到)
        messages.append({"role": "assistant", "content": assistant_response})
        try:
            memory.add(messages, user_id=user_id)
        except Exception as add_err:
            print(f"Warning: 记忆写入失败，下次可能无法记住本次对话: {add_err}")

        return assistant_response
        
    except Exception as e:
        print(f"Error in chat_with_memories: {e}")
        # Fallback response without memory
        messages = [{"role": "user", "content": message}]
        response = openai_client.chat.completions.create(model="gpt-4o", messages=messages)
        return response.choices[0].message.content

def main():
    """Interactive chat loop (for terminal; in Jupyter use chat_with_memories('your question') in a cell)"""
    print("Chat with AI (type 'exit' to quit)")
    print("This AI has memory and can remember your conversation!")
    try:
        while True:
            user_input = input("\nYou: ").strip()
            if user_input.lower() == 'exit':
                print("Goodbye!")
                break
            try:
                response = chat_with_memories(user_input)
                print(f"AI: {response}")
            except Exception as e:
                print(f"Error: {e}")
    except Exception as e:
        if "StdinNotImplementedError" in type(e).__name__ or "input" in str(e).lower():
            print("Notebook: run chat_with_memories('your question') in a cell instead.")
        else:
            raise

if __name__ == "__main__":
    main()

Chat with AI (type 'exit' to quit)
This AI has memory and can remember your conversation!


INFO:mem0.vector_stores.mongodb:Retrieved document with ID 'c0c3c749-98eb-4d92-9620-8283735e0a66'.
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 0 documents.
INFO:mem0.vector_stores.mongodb:Retrieved document with ID 'c0c3c749-98eb-4d92-9620-8283735e0a66'.
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 3 documents.


Memories:
No relevant memories found.
Semantic Memories:
- [140] Dacheng Li, Rulin Shao, Anze Xie, Ying Sheng, Lianmin Zheng, Joseph Gonzalez, Ion Stoica,
Xuezhe Ma, and Hao Zhang. How long can context length of open-source llms truly promise?
In NeurIPS 2023 Workshop on Instruction Tuning and Instruction Following, 2023.
[141] Mohit Shridhar, Xingdi Yuan, Marc-Alexandre Côté, Yonatan Bisk, Adam Trischler, and
Matthew Hausknecht. Alfworld: Aligning text and embodied environments for interactive
learning. arXiv preprint arXiv:2010.03768, 2020.
[142] YunDa Tsai, Mingjie Liu, and Haoxing Ren. Rtlfixer: Automatically fixing rtl syntax errors
with large language models. arXiv preprint arXiv:2311.16543, 2023.
[143] Cheng Li, Ziang Leng, Chenxi Yan, Junyi Shen, Hao Wang, Weishi Mi, Yaying Fei, Xiaoyang
Feng, Song Yan, HaoSheng Wang, et al. Chatharuhi: Reviving anime character in reality via
large language model. arXiv preprint arXiv:2308.09597, 2023.
[144] Dake Chen, Hanbin Wang, Yunhao Huo, 

INFO:httpx:HTTP Request: POST https://newenglandhackopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://newenglandhackopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
INFO:mem0.memory.main:Total existing memories: 0
INFO:mem0.vector_stores.mongodb:Retrieved document with ID 'c0c3c749-98eb-4d92-9620-8283735e0a66'.


AI: Could you clarify your inquiry? Are you referring to a number, a context, or something specific?


...

INFO:mem0.vector_stores.mongodb:Retrieved document with ID 'c0c3c749-98eb-4d92-9620-8283735e0a66'.
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 1 documents.
INFO:mem0.vector_stores.mongodb:Retrieved document with ID 'c0c3c749-98eb-4d92-9620-8283735e0a66'.
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 3 documents.


Memories:
- 用户的名字是user001
Semantic Memories:
- easily exceed the upper bound of the sequence length during LLM’s pretraining, which makes a
truncation of memory necessary. Thus, it can lead to information loss due to the incompleteness
of agent memory. Last but not least, it can lead to biases and unrobustness in LLM’s inference.
Specifically, a previous research [ 120] has shown that, the positions of text segments in a long
context can greatly affect their utilization, so the memory in the long-context prompt can not be
treated equally and stably. All these drawbacks show the need to design extra memory modules for
LLM-based agents, rather than straightforwardly concatenating all the information into a prompt.
Recent Interactions. This method stores and maintains the most recently acquired memories using
natural languages, thereby enhancing the efficiency of memory information utilization according
to the Principle of Locality [121]. In task (B) of the example in Section 3.1, we can 

INFO:httpx:HTTP Request: POST https://newenglandhackopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://newenglandhackopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 1 documents.
INFO:mem0.memory.main:Total existing memories: 1
INFO:httpx:HTTP Request: POST https://newenglandhackopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
INFO:mem0.memory.main:{'id': '0', 'text': '用户的名字是user001', 'event': 'NONE'}
INFO:mem0.memory.main:NOOP for Memory.
INFO:mem0.vector_stores.mongodb:Retrieved document with ID 'c0c3c749-98eb-4d92-9620-8283735e0a66'.


AI: 你并未在之前的对话中留下具体的问题记录，只是涉及了一些关于记忆模块和LLM的讨论。如果需要，你可以重述你的问题，我会尽力回答！
AI: Please enter a message.
AI: Please enter a message.


KeyboardInterrupt: Interrupted by user

In [33]:
coffee_search = memory.search(query="coffee brewing methods and preferences", user_id=CURRENT_USER_ID, limit=5)


INFO:mem0.vector_stores.mongodb:Retrieved document with ID 'c0c3c749-98eb-4d92-9620-8283735e0a66'.
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 1 documents.


In [34]:
coffee_search

{'results': [{'updated_at': None,
   'hash': None,
   'user_id': 'user-123',
   'agent_id': None,
   'data': '用户的名字是user001',
   'role': None,
   'created_at': None,
   'actor_id': None,
   'run_id': None,
   'id': '200770eb-98a6-403d-836b-8dcdb46c3f90',
   'score': 0.614314079284668,
   'memory': '用户的名字是user001',
   'metadata': {}}]}

In [35]:
paper_search = semantic_memory.search(query="What is the main idea of the paper?", user_id=CURRENT_USER_ID, limit=5)

INFO:mem0.vector_stores.mongodb:Retrieved document with ID 'c0c3c749-98eb-4d92-9620-8283735e0a66'.
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 5 documents.


In [36]:
paper_search

{'results': [{'updated_at': None,
   'hash': None,
   'user_id': 'user-123',
   'agent_id': None,
   'data': 'be stored in the same group. In SCM [98], it designs a memory controller to decide when to execute\nthe operations. The controller serves as a guide for the whole memory module. In MemGPT [100],\nthe memory writing is entirely self-directed. The agents can autonomously update the memory based\non the contexts. In MemoChat [94], the agents summarize each conversation segment by abstracting\nthe mainly discussed topics and storing them as keys for indexing memory pieces.\nDiscussion. Previous research indicates that designing the strategy of information extraction during\nthe memory writing operation is vital [94]. This is because the original information is commonly\nlengthy and noisy. Besides, different environments may provide various forms of feedback, and how\nto extract and represent the information as memory is also significant for memory writing.\n5.3.2 Memory Management\

In [43]:
chat_with_memories("我叫什么名字？我喜欢吃什么水果？",user_id=CURRENT_USER_ID)


INFO:mem0.vector_stores.mongodb:Retrieved document with ID 'c0c3c749-98eb-4d92-9620-8283735e0a66'.
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 3 documents.
INFO:mem0.vector_stores.mongodb:Retrieved document with ID 'c0c3c749-98eb-4d92-9620-8283735e0a66'.
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 3 documents.


Memories:
- 用户喜欢吃苹果
- 名字叫周斌
- 用户的名字是user001
Semantic Memories:
- You may like this blue dress. It is 
of good quality and great price.
Would you like to buy a 
waistband for your dress?
Great! I like this bule one. 
I will buy it for the party.
Medicine
David, a 38-year-old male with a history of allergies and sinus 
infections, has a family history of diabetes and hypertension. As 
a smoker of about one pack a day and an occasional drinker, 
his lifestyle choices may contribute to his health risks. After 
traveling to a tropical country where mosquito-borne illnesses 
are prevalent, he has experienced symptoms such as mild 
fatigue, headache, and muscle aches for the past week. 
For three days, I’ve had a 
fever ranging from 
100.5°F to 102°F, a rash 
on my limbs, joint pain 
and swelling, especially 
in my hands, episodes of 
diarrhea, abdominal pain, 
and nausea, leading to a 
loss of appetite.
The patient‘s travel history and 
symptoms suggest dengue fever, 
a mosquito-transmitted 

INFO:httpx:HTTP Request: POST https://newenglandhackopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://newenglandhackopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 3 documents.
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 3 documents.
INFO:mem0.memory.main:Total existing memories: 3
INFO:httpx:HTTP Request: POST https://newenglandhackopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
INFO:mem0.memory.main:{'id': '0', 'text': '用户的名字是周斌', 'event': 'UPDATE', 'old_memory': '名字叫周斌'}
INFO:mem0.memory.main:Updating memory with data='用户的名字是周斌'
INFO:mem0.vector_stores.mongodb:Retrieved document with ID 'c7a1c439-a0c8-4674-be3b-4f90f4363990'.
INFO:mem0.vector_stores.mongodb:Updated d

'你叫周斌，你喜欢吃苹果。'